In [ ]:
'''
參考網址
1. http://free-proxy.cz/zh/proxylist/country/US/https/uptime/all
2. https://ip.seeip.org/jsonip
3. https://pypi.org/project/fake-useragent/
'''


import requests as req
from bs4 import BeautifulSoup as bs
import re, base64
from random import randint
from pprint import pprint

# 隨機取得 User-Agent
from fake_useragent import UserAgent
ua = UserAgent(cache=True) # cache=True 表示從已經儲存的列表中提取

# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep, time

In [ ]:
# 欲抓取資料的網址
domainName = 'https://www.wine-searcher.com'
url = domainName + '/find/mouton+rothschild+pauillac+medoc+bordeaux+france'

# proxy 設定
my_proxies = {
    "http": "",
    "https": "",
}

# 放置 proxy 列表的變數，每次用完或確認無法使用，就會拋棄，用完了，會再重新請求
listProxyPool = []

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文
# my_options.add_argument(f'--proxy-server={ip_port}') # web driver 也可以設定 proxy

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)

In [ ]:
'''
函式
'''
# 取得 free proxy 列表 (http://free-proxy.cz/en/)
def getProxyPool():
    global listProxyPool
    global my_proxies
    global driver
    
    # 若先前有 proxy pool 資料，則先清空
    listProxyPool = []
    
    # 暫存 ip 列表
    listTmp = []
    
    try:
        # 開始時間
        time_begin = time()
        
        # 走訪分頁
        for page in range(1, 4):
            # 隨機等待
            sleep(randint(2, 5))
            
            # 取得 html
            driver.get(f'http://free-proxy.cz/zh/proxylist/country/all/https/ping/all/{page}')

            # 等待元素出現
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "table#proxy_list > tbody > tr")
                )
            )

            # 建立 soup
            soup = bs(driver.page_source, "lxml")

            # 選擇 table 裡面的所有 tr
            tr_elms = soup.select('table#proxy_list > tbody > tr')

            # 定義 regex，準備對 ip 進行解碼
            regexIP = r"Base64\.decode\(\"(.+?)\"\)"

            # 走訪所有 tr
            for tr in tr_elms:
                # 找出 tr 當中的所有 td
                td_elms = tr.select('td')

                # 對 ip 解碼
                ip = str(td_elms[0])
                match = re.search(regexIP, ip)
                if match == None:
                    continue
                ip = match[1]
                ip = base64.b64decode(ip)
                ip = ip.decode("utf-8")

                # 找出 port
                port = td_elms[1].get_text().strip()

                # 整理可用資料
                listTmp.append(f"{ip}:{port}")
            
        # 目前有多少 ip 可進行測試
        print(f"測試 IP 數: {len(listTmp)}")

        # 過濾掉當前不可用的 ip:port
        for ip in listTmp:
            try:
                # 設定 proxy
                my_proxies['http'] = ip
                my_proxies['https'] = ip
                
                # 發出請求，取得回應
                res = req.get('https://ip.seeip.org/jsonip', proxies = my_proxies, timeout = 2)
                
                # 回傳 200 ok，代表 ip 暫時可用
                if res.ok:
                    print(f"可用 IP: {res.json()}")

                    # 將可用 ip 加入 proxy pool
                    listProxyPool.append(ip)
                else:
                    print(f"{ip} 無法使用...")
            except:
                print(f"{ip} 無法使用...")

        print(f"可用 IP 列表 ({len(listProxyPool)}):")
        pprint(listProxyPool)

        # 計算結束時間
        print(f"取得可用 IP 列表花費 {time() - time_begin} 秒")
    except TimeoutException:
        print("等待逾時...")
    
# 取得首頁列表資訊
def checkProxy():
    global listProxyPool
    global my_proxies
    
    print("=" * 15 + "開始測試請求" + "=" * 15)
    
    # 逐個 ip 測試
    for ip in listProxyPool:
        try:
            print(f"測試 IP: {ip}")
            
            # 一個 IP 測 5 次，看看是不是真的可以使用
            for i in range(5):
                # 設定 proxy
                my_proxies['http'] = ip
                my_proxies['https'] = ip
                
                # 設定隨機請求標頭
                my_headers = {'user-agent': ua.random}
                
                # 請求頁面
                res = req.get(url, headers = my_headers, proxies = my_proxies, timeout = 2)
                
                # 回傳 200 ok，代表 ip 暫時可用
                if res.ok:
                    # 簡單確認回傳結果，若包括 automation 字眼，代表可能被擋，要換 ip
                    regex = r"Access to this page has been denied"
                    match = re.search(regex, res.text)
                    if match != None:
                        print(f"無法使用...")
                        break
                    else:
                        print("***結果: 成功，請求 (request) 已得到回應 (response)***")
                        print(f"當前使用 Proxy: {my_proxies}")
                        sleep( randint(2,5) )
                else:
                    print(f"第 {i + 1} 次 請求失敗")
        except:
            print(f"該 IP 無法使用...")
            continue

In [ ]:
'''
主程式
'''
if __name__ == "__main__":
    time_begin = time() # 開始時間
    getProxyPool()
    checkProxy()
    print(f"執行花費 {time() - time_begin} 秒")